In [31]:
import pandas as pd
import numpy as np
import bitcoin
import datetime
import random

import sys
sys.path.append("..")
from Network_delay.delayDistribution import randomDelay, randomDelayList, user_delay_normal_09

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

%pwd

'h:\\Codes\\PythonWorkspace\\CSE5005-Project\\demo'

In [32]:
def gen_addr():
    # 生成一个随机的密钥
    while True:
        # 生成一个用十六进制表示的长 256 位的私钥（str类型）
        private_key = bitcoin.random_key()
        # 解码为十进制的整形密钥
        decoded_private_key = bitcoin.decode_privkey(private_key, "hex")
        if 0 < decoded_private_key < bitcoin.N:
            break
    # print(private_key) #密钥（十六进制,长 256 位）

    # 计算地址
    # 用 WIF 格式编码密钥
    wif_encoded_private_key = bitcoin.encode_privkey(decoded_private_key, "wif")
    # 用 01 标识的压缩密钥
    compressed_private_key = private_key + "01"
    # 生成 WIF的压缩格式
    wif_compressed_private_key = bitcoin.encode_privkey(
        bitcoin.decode_privkey(compressed_private_key, "hex"), "wif"
    )
    # 计算公钥坐标 K = k * G
    public_key = bitcoin.fast_multiply(bitcoin.G, decoded_private_key)
    # 计算公钥
    hex_encoded_public_key = bitcoin.encode_pubkey(public_key, "hex")
    # 计算压缩公钥
    # if public_key[1] % 2 == 0:  # 两种方式均可
    if public_key[1] & 1 == 0:
        compressed_prefix = "02"
    else:
        compressed_prefix = "03"
    # 转十六也可用 bitcoin.encode(xxx, 16)
    hex_compressed_public_key = compressed_prefix + hex(public_key[0])[2:]
    # print(f'压缩公钥（十六进制）{hex_compressed_public_key} '
    #      '（02 开头代表 y 是偶数，03 开头代表 y 是奇数）')
    # 传入公钥坐标对象/十六进制公钥值，输出同样的地址
    address = bitcoin.pubkey_to_address(public_key)
    # print(address) #地址（b58check）（1 开头
    return address

In [33]:
num_user=500

df_user = pd.DataFrame(columns=["user_id", "address"])
for i in range(num_user):
    df_user.loc[i] = [i, gen_addr()]

df_user["mean"]=randomDelayList(num_user)
df_user

,user_id,address,mean
0,0,1Ezzto898LdNa7aijuByjkRbgUbpa3NEFg,132.776393
1,1,1A1xJqpBTnE6kXWrkjmtSKDtxMY9sS19FP,86.586443
2,2,1NQGdTLwPfsgthJ4vjQq2Z6ba9igXxmWdP,108.590557
3,3,15QzFcs4LUJbEcDFt1GPgTzj6c8MsG7rHh,92.245653
4,4,17Sw92TxGD3Q4UZuCLCJpsG31nuTSgWNf2,163.863143
...,...,...,...
495,495,1D9hw8xDkPRf5Fn4DtUG3e8RmJv7rJJLyc,10.158448
496,496,17PvpdTm5NdStrAtfCn6jAhBc9udhcZtHe,70.044583
497,497,1PBBCywffXkTT1gfztvpKFqo4G9X4xGXVX,89.850344
498,498,1FFLCS655UG7h4F3p7w3tJR6jTvaqRiPYN,84.144983


In [34]:
df_user.to_csv("temp_user.csv", index=False)

In [35]:
df_tx = pd.DataFrame(
        columns=[
            "send_timestamp",
            "recv_timestamp",
            "latency",
            "label",
            "from",
            "to",
            "value",
        ]
    )
    
n=5000
for _ in range(n):
    send_ts = datetime.datetime.now()
    
    from_user_idx=random.randint(0, len(df_user) - 1)
    mean=df_user.loc[from_user_idx, "mean"]
    
    latency=user_delay_normal_09(mean, range=mean/10)
    fluc = randomDelay()
    latency = 0.7 * latency + 0.3 * fluc
    
    recv_ts = send_ts + datetime.timedelta(seconds=latency)
    
    from_user = df_user.loc[from_user_idx, "address"]
    to_user = np.random.choice(df_user["address"].values)
    value = random.random() * 10
    label = 0
    
    df_tx.loc[len(df_tx)]=[send_ts, recv_ts, latency, label, from_user, to_user, value]
    
df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-12-20 18:17:34.485413,2022-12-20 18:19:12.465123,97.979710,0,1MSa2dfnTgWRwdS7akjasH5cVbfCz29EZZ,13QehqQB3EavEToRxtnXcZ9t5aUm3aCdXE,1.021071
1,2022-12-20 18:17:34.487407,2022-12-20 18:20:03.059867,148.572460,0,15bkCmDRCwbbucAnbmxCM7zUpe27aUGV5a,1NXKmt9szwQEtU9WEgGzSyGeWRr84cnHHV,4.191815
2,2022-12-20 18:17:34.492394,2022-12-20 18:20:40.433076,185.940682,0,13KGnpWbLU7yJmu6mkfwvBRQffVPgwDYr9,1PEWb7j2WpE8mZmSxcbRsXM9wEqDg9CAjf,6.963535
3,2022-12-20 18:17:34.496418,2022-12-20 18:20:12.930596,158.434178,0,1LTUZ9nneZ4ai2wxutaGF4ciJna3wCLwBe,1GCrxbzYr6a3wT9HGNvCcCYvW2sDT44vAe,6.148985
4,2022-12-20 18:17:34.499376,2022-12-20 18:19:29.704551,115.205175,0,1LkAkwf7pnNX9iW2FqJ7wDYvwkr2Hv1WBq,1Hk9uwt2rJjECvbdEwE4fhGCy2DNgBd86,3.296789
...,...,...,...,...,...,...,...
4995,2022-12-20 18:17:45.944084,2022-12-20 18:17:58.858917,12.914833,0,1KoRhkUj12nrddw41iDXj1tQSMHZAVjAcH,14htnGJq3RiV4fV2t7dLWSdvKdtWt4Qu69,6.211750
4996,2022-12-20 18:17:45.946078,2022-12-20 18:20:35.557217,169.611139,0,1EUAk62bEbyuSeLipb5yuXDon6wZ92KWmp,1LMHBLRpvwf8s6Nc5GKY2QDnmAwKApgaTV,0.694297
4997,2022-12-20 18:17:45.948073,2022-12-20 18:19:00.775882,74.827809,0,1Gnyj18vDx8visBio6PtiPjk1sCPry2NhK,1NBpUy76e2SMKFKaXtAr65msFetjTPVLKn,0.112733
4998,2022-12-20 18:17:45.951066,2022-12-20 18:20:25.877355,159.926289,0,164xoGoCX32FFFDWatKD7krVYD1FAc5Eic,1B6q1fziCwRaBAswLSnCv554LRfbBnjdJE,3.068479


In [36]:
df_tx.sort_values(by="recv_timestamp", inplace=True)

last_send_ts=datetime.datetime(1, 1, 1, 1, 1, 1)
for index, row in df_tx.iterrows():
    if row["send_timestamp"]<last_send_ts:
        df_tx.loc[index, "label"]=1
    last_send_ts=row["send_timestamp"]
    
df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
433,2022-12-20 18:17:35.418461,2022-12-20 18:17:42.289950,6.871489,0,1PGvY76CgJMSze1V48GZtRYA4K9kW4FnuD,1Q8kU6utjSvFX5ihxp672vubFs19R3TYaN,4.536026
368,2022-12-20 18:17:35.285525,2022-12-20 18:17:46.213868,10.928343,1,1PGvY76CgJMSze1V48GZtRYA4K9kW4FnuD,1PjLGcAEfR2otEpnSbZosigw6f3D7CHzN4,8.904795
2305,2022-12-20 18:17:39.633891,2022-12-20 18:17:49.359392,9.725501,0,1DUY2zzNFnmJbdCgytVmG2C6BswFM6P6gg,19voNhWY8vjk2NoBUU4L8wcBjzFyCxodqm,1.933107
2512,2022-12-20 18:17:40.131160,2022-12-20 18:17:49.887297,9.756137,0,1KoRhkUj12nrddw41iDXj1tQSMHZAVjAcH,19mFjo1UpYePKgkmvg8s3MbAQztqGYm8cu,9.643031
2447,2022-12-20 18:17:39.974757,2022-12-20 18:17:49.906923,9.932166,1,1PGvY76CgJMSze1V48GZtRYA4K9kW4FnuD,1HToxmkkWrZxHWn4YCrReMVRn8Nmk31pF1,3.569513
...,...,...,...,...,...,...,...
4680,2022-12-20 18:17:45.065585,2022-12-20 18:23:12.932648,327.867063,0,1EuLBroJ3AgtLYpsA2AGaMoU8WmuCKZccs,1Gnyj18vDx8visBio6PtiPjk1sCPry2NhK,2.816094
1039,2022-12-20 18:17:36.686598,2022-12-20 18:23:14.280656,337.594058,1,18h1WQWqcq7xYfoVXRbUQmnSxydrj76ufB,1GQFBpj3UcAkVLPEMwVK1h8s3Fgok5thmw,7.857793
3898,2022-12-20 18:17:43.316541,2022-12-20 18:23:24.123623,340.807082,0,18h1WQWqcq7xYfoVXRbUQmnSxydrj76ufB,1KT5y3DukrKrL4pbarjiEe5R32UVH1eSuo,8.677200
3977,2022-12-20 18:17:43.497766,2022-12-20 18:23:24.286253,340.788487,0,18h1WQWqcq7xYfoVXRbUQmnSxydrj76ufB,1QACHqmZN9w6uE1YfJkXyJQPvi87AuKdmj,6.413028


In [37]:
sum(df_tx["label"] == 1)

2505

In [38]:
user_list=df_user["address"].values.tolist()

def get_user_id(row):
    from_user_id=user_list.index(row["from"])
    to_user_id=user_list.index(row["to"])
    
    row["from_user_id"]=from_user_id
    row["to_user_id"]=to_user_id
    
    return row

In [39]:
df_data=df_tx.copy()
df_data=df_data.apply(get_user_id, axis=1)

df_data.drop(columns=["send_timestamp", "from", "to"], inplace=True)

df_data

,recv_timestamp,latency,label,value,from_user_id,to_user_id
433,2022-12-20 18:17:42.289950,6.871489,0,4.536026,38,244
368,2022-12-20 18:17:46.213868,10.928343,1,8.904795,38,41
2305,2022-12-20 18:17:49.359392,9.725501,0,1.933107,368,246
2512,2022-12-20 18:17:49.887297,9.756137,0,9.643031,14,164
2447,2022-12-20 18:17:49.906923,9.932166,1,3.569513,38,225
...,...,...,...,...,...,...
4680,2022-12-20 18:23:12.932648,327.867063,0,2.816094,47,96
1039,2022-12-20 18:23:14.280656,337.594058,1,7.857793,280,454
3898,2022-12-20 18:23:24.123623,340.807082,0,8.677200,280,336
3977,2022-12-20 18:23:24.286253,340.788487,0,6.413028,280,269


In [40]:
data_path="temp_data.pkl"

df_data.to_pickle(data_path)

In [41]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

def onehot_decode(label):
    return torch.argmax(label, dim=1)

def accuracy(predictions, targets):
    pred_decode = onehot_decode(predictions)
    true_decode = targets

    assert len(pred_decode) == len(true_decode)

    acc = torch.mean((pred_decode == true_decode).float())

    return float(acc)

class BinaryClassifier(nn.Module):
    def __init__(
        self,
        num_users,
        embedding_dim=16,
        hidden_dim=32,
    ) -> None:
        super().__init__()

        self.num_users = num_users
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        self.input_dim = embedding_dim * 2 + 1

        self.user_embedding_layer = nn.Embedding(
            num_embeddings=num_users, embedding_dim=embedding_dim, padding_idx=-1
        )

        self.mlp = nn.Linear(self.input_dim, 2)

    def forward(self, x):
        # x: (batch_size, 2, 3(from, to, label))

        last_tx = x[:, 0, :]
        cur_tx = x[:, 1, :]

        last_from_user = last_tx[:, 0].long()
        # last_to_user = last_tx[:, 1].long()
        last_label = last_tx[:, 2][:, None]

        cur_from_user = cur_tx[:, 0].long()
        # cur_to_user = cur_tx[:, 1].long()

        last_from_embedding = self.user_embedding_layer(
            last_from_user
        )  # (batch_size, embedding_dim)
        # last_to_embedding = self.user_embedding_layer(last_to_user)

        cur_from_embedding = self.user_embedding_layer(cur_from_user)
        # cur_to_embedding = self.user_embedding_layer(cur_to_user)

        input = torch.concat(
            (
                last_from_embedding,
                # last_to_embedding,
                cur_from_embedding,
                # cur_to_embedding,
                last_label,
            ),
            dim=1,
        )

        out = self.mlp(input)
        out = torch.softmax(out, dim=1)

        return out
    
    def get_embeddings(self):
        return self.user_embedding_layer.weight.cpu().detach().numpy()

def gen_xy(sequence):
    """
    Returns
    ---
    x: (num_samples, 2, num_features)
    y: (num_samples,) 1-d vec for labels
    """

    x, y = [], []
    for i in range(len(sequence) - 2):
        x.append(sequence[i : i + 2])
        y.append(sequence[i + 2 - 1])

    x = np.array(x)
    y = np.array(y)

    if len(y.shape) > 1:
        y = y[:, -1]

    return torch.FloatTensor(x), torch.LongTensor(y)


def get_dataloaders(sequence, train_size=0.7, val_size=0.1, batch_size=64):
    x, y = gen_xy(sequence)

    split1 = int(len(x) * train_size)
    split2 = int(len(sequence) * (train_size + val_size))

    x_train, y_train = x[:split1], y[:split1]
    x_val, y_val = x[split1:split2], y[split1:split2]
    x_test, y_test = x[split2:], y[split2:]

    print(f"Trainset:\tx-{x_train.size()}\ty-{y_train.size()}")
    print(f"Valset:  \tx-{x_val.size()}  \ty-{y_val.size()}")
    print(f"Testset:\tx-{x_test.size()}\ty-{y_test.size()}")

    trainset = torch.utils.data.TensorDataset(x_train, y_train)
    valset = torch.utils.data.TensorDataset(x_val, y_val)
    testset = torch.utils.data.TensorDataset(x_test, y_test)

    trainset_loader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=True
    )
    valset_loader = torch.utils.data.DataLoader(
        valset, batch_size=batch_size, shuffle=True
    )
    testset_loader = torch.utils.data.DataLoader(
        testset, batch_size=batch_size, shuffle=False
    )

    return trainset_loader, valset_loader, testset_loader


@torch.no_grad()
def eval_model(model, valset_loader, criterion):
    model.eval()
    batch_loss_list = []
    batch_acc_list = []
    for x_batch, y_batch in valset_loader:
        x_batch = x_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)

        out_batch = model.forward(x_batch)
        loss = criterion.forward(out_batch, y_batch)
        batch_loss_list.append(loss.item())

        acc = accuracy(out_batch, y_batch)
        batch_acc_list.append(acc)

    return np.mean(batch_loss_list), np.mean(batch_acc_list)


def train_one_epoch(model, trainset_loader, optimizer, criterion):
    model.train()
    batch_loss_list = []
    batch_acc_list = []
    for x_batch, y_batch in trainset_loader:
        x_batch = x_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)

        out_batch = model.forward(x_batch)
        loss = criterion.forward(out_batch, y_batch)
        batch_loss_list.append(loss.item())

        acc = accuracy(out_batch, y_batch)
        batch_acc_list.append(acc)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(batch_loss_list), np.mean(batch_acc_list)


def train(
    model,
    trainset_loader,
    valset_loader,
    optimizer,
    criterion,
    max_epochs=100,
    early_stop=10,
    verbose=1,
    plot=False,
):
    wait = 0
    min_val_loss = np.inf

    train_loss_list = []
    train_acc_list = []
    val_loss_list = []
    val_acc_list = []

    for epoch in range(max_epochs):
        train_loss, train_acc = train_one_epoch(
            model, trainset_loader, optimizer, criterion
        )
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)

        val_loss, val_acc = eval_model(model, valset_loader, criterion)
        val_loss_list.append(val_loss)
        val_acc_list.append(val_acc)

        if (epoch + 1) % verbose == 0:
            print(
                datetime.datetime.now(),
                "Epoch",
                epoch + 1,
                "\tTrain Loss = %.5f" % train_loss,
                "Train acc = %.5f " % train_acc,
                "Val Loss = %.5f" % val_loss,
                "Val acc = %.5f " % val_acc,
            )

        if val_loss < min_val_loss:
            wait = 0
            min_val_loss = val_loss
            best_epoch = epoch
            best_state_dict = model.state_dict()
        else:
            wait += 1
            if wait >= early_stop:
                print(f"Early stopping at epoch: {epoch+1}")
                print(f"Best at epoch {best_epoch+1}:")
                print(
                    "Train Loss = %.5f" % train_loss_list[best_epoch],
                    "Train acc = %.5f " % train_acc_list[best_epoch],
                )
                print(
                    "Val Loss = %.5f" % val_loss_list[best_epoch],
                    "Val acc = %.5f " % val_acc_list[best_epoch],
                )

                break

    if plot:
        plt.plot(range(0, epoch + 1), train_loss_list, "-", label="Train Loss")
        plt.plot(range(0, epoch + 1), val_loss_list, "-", label="Val Loss")
        plt.title("Epoch-Loss")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.show()

        plt.plot(range(0, epoch + 1), train_acc_list, "-", label="Train Acc")
        plt.plot(range(0, epoch + 1), val_acc_list, "-", label="Val Acc")
        plt.title("Epoch-Accuracy")
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.show()

    model.load_state_dict(best_state_dict)
    return model


if __name__ == "__main__":
    num_users = 500
    embedding_dim = 8
    hidden_dim = 16
    batch_size = 64
    lr = 1e-4

    max_epochs = 1000

    if torch.cuda.is_available():
        GPU_ID = 0
        DEVICE = torch.device(f"cuda:{GPU_ID}")
    else:
        DEVICE = torch.device("cpu")
        
    sequence = pd.read_pickle(data_path)[
        ["from_user_id", "to_user_id", "label"]
    ].values

    train_loader, val_loader, test_loader = get_dataloaders(
        sequence, batch_size=batch_size, train_size=0.7, val_size=0.1
    )

    model = BinaryClassifier(
        num_users=num_users, embedding_dim=embedding_dim, hidden_dim=hidden_dim
    ).to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model = train(
        model,
        train_loader,
        val_loader,
        optimizer,
        criterion,
        max_epochs=max_epochs,
        early_stop=10,
        verbose=1,
        plot=False,
    )

    test_loss, test_acc = eval_model(model, test_loader, criterion)
    print("Test Loss = %.5f" % test_loss, "Test acc = %.5f " % test_acc)
    

Trainset:	x-torch.Size([3498, 2, 3])	y-torch.Size([3498])
Valset:  	x-torch.Size([501, 2, 3])  	y-torch.Size([501])
Testset:	x-torch.Size([999, 2, 3])	y-torch.Size([999])
2022-12-20 18:17:52.389777 Epoch 1 	Train Loss = 0.70182 Train acc = 0.52050  Val Loss = 0.72016 Val acc = 0.48596 
2022-12-20 18:17:52.618163 Epoch 2 	Train Loss = 0.70119 Train acc = 0.51989  Val Loss = 0.72030 Val acc = 0.48157 
2022-12-20 18:17:52.794691 Epoch 3 	Train Loss = 0.70037 Train acc = 0.52116  Val Loss = 0.71880 Val acc = 0.48239 
2022-12-20 18:17:52.992162 Epoch 4 	Train Loss = 0.69900 Train acc = 0.52407  Val Loss = 0.71816 Val acc = 0.48157 
2022-12-20 18:17:53.147756 Epoch 5 	Train Loss = 0.69838 Train acc = 0.52574  Val Loss = 0.71678 Val acc = 0.48677 
2022-12-20 18:17:53.350335 Epoch 6 	Train Loss = 0.69793 Train acc = 0.52727  Val Loss = 0.71597 Val acc = 0.48791 
2022-12-20 18:17:53.504920 Epoch 7 	Train Loss = 0.69692 Train acc = 0.52871  Val Loss = 0.71561 Val acc = 0.48629 
2022-12-20 18:17: